# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846836510189                   -0.70    4.5         
  2   -7.852320102264       -2.26       -1.53    1.0   22.3ms
  3   -7.852615149103       -3.53       -2.56    1.5   24.8ms
  4   -7.852645988885       -4.51       -2.88    2.5   31.1ms
  5   -7.852646497321       -6.29       -3.16    1.0   22.8ms
  6   -7.852646678499       -6.74       -3.97    1.0   23.2ms
  7   -7.852646686362       -8.10       -5.08    1.5   26.2ms
  8   -7.852646686722       -9.44       -5.39    2.5   31.4ms
  9   -7.852646686729      -11.16       -5.94    1.0   23.1ms
 10   -7.852646686730      -12.23       -6.32    1.2   25.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846711339708                   -0.70           4.8         
  2   -7.852551417790       -2.23       -1.64   0.80    2.0    251ms
  3   -7.852637516473       -4.07       -2.76   0.80    1.0   21.8ms
  4   -7.852646559659       -5.04       -3.30   0.80    2.0   27.1ms
  5   -7.852646676212       -6.93       -4.15   0.80    1.0   21.2ms
  6   -7.852646686350       -7.99       -4.83   0.80    1.5   24.3ms
  7   -7.852646686720       -9.43       -5.82   0.80    1.8   25.2ms
  8   -7.852646686730      -11.00       -6.61   0.80    2.5   28.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.392974e+01     3.658807e+00
 * time: 0.0574040412902832
     1     1.006004e+00     1.888416e+00
 * time: 0.26959896087646484
     2    -1.793979e+00     2.055529e+00
 * time: 0.29210805892944336
     3    -3.781946e+00     1.724610e+00
 * time: 0.3252260684967041
     4    -5.340734e+00     1.336223e+00
 * time: 0.3573930263519287
     5    -6.910200e+00     8.413963e-01
 * time: 0.38973212242126465
     6    -7.122929e+00     1.181162e+00
 * time: 0.41250109672546387
     7    -7.633963e+00     8.734192e-01
 * time: 0.43518805503845215
     8    -7.698615e+00     1.257339e+00
 * time: 0.45757603645324707
     9    -7.724728e+00     9.670677e-01
 * time: 0.4801759719848633
    10    -7.763006e+00     4.636726e-01
 * time: 0.512455940246582
    11    -7.795849e+00     6.360716e-01
 * time: 0.5352420806884766
    12    -7.816957e+00     1.114486e-01
 * time: 0.5577669143676758
    13    -7.829783e+00     9.277646e-02
 * time: 0.58046

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846903803553                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645923303                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.84s
  3   -7.852646686730      -13.30       -7.26    185ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.0753162055331485e-7
|ρ_newton - ρ_scfv| = 1.3879863813519674e-7
|ρ_newton - ρ_dm|   = 9.674000561725078e-10
